In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import pickle

import fiona
from itertools import chain
from itertools import product

import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from mpl_toolkits.basemap import Basemap
from shapely.geometry import Point, Polygon, MultiPoint
from descartes import PolygonPatch

import matplotlib.colors as mpl_colors
from random import randint
import time

from geopy.distance import vincenty

from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

In [ ]:
threshold_dist_miles = 0.2

with open('data_routes_pickle/routes_coord', 'rb') as f:
    routes_coord = pickle.load(f)

coord_list = [[num for coords in route for num in coords] for route in routes_coord]

def dist_vinc(pair):
    return vincenty(pair[0], pair[1]).miles

def compute_distance(x, y, metric='l1'):
    if metric == 'l1':
        return np.linalg.norm((np.array(x) - np.array(y)), ord=1)
    elif metric == 'sim_points':
        points_x = list(zip(x[::2], x[1::2]))
        points_y = list(zip(y[::2], y[1::2]))
        
        total_common_count = 0
        most_sim_points = min(product(points_x, points_y), key=dist_vinc)
        
        while (vincenty(most_sim_points[0], most_sim_points[1]).miles < threshold_dist_miles and
               len(points_x) > 1):
            total_common_count += 1
            
            points_x.remove(most_sim_points[0])
            points_y.remove(most_sim_points[1])
                
            most_sim_points = min(product(points_x, points_y), key=dist_vinc)
            
        return 96 - total_common_count * 2
    else:
        raise Exception('Unknown metric')
        
# number_of_paths = len(coord_list)
number_of_paths = 150 - 80
distance_matrix = np.zeros((number_of_paths, number_of_paths))
for i in range(number_of_paths):
    for j in range(i, number_of_paths):
        distance = compute_distance(coord_list[i + 80], coord_list[j + 80], 'sim_points')
#         distance = compute_distance(coord_list[i], coord_list[j], 'l1')
        distance_matrix[i][j] = distance
        distance_matrix[j][i] = distance
        print('Elem done')
    print('Row done')

In [ ]:
distance_matrix /= 6

In [ ]:
epsilon_range = np.arange(0.25, 1.55, 0.25) # y
min_sample_range = np.arange(1, 10, 1) # x
quality_matrix = np.zeros((len(epsilon_range), len(min_sample_range)))
cluster_count_matrix = np.zeros((len(epsilon_range), len(min_sample_range)))

# clustering_algorithm = DBSCAN(eps=0.5, min_samples=1, metric='precomputed')
# labels = clustering_algorithm.fit_predict(distance_matrix)
# sil = silhouette_score(np.array(coord_list[80:150]), labels)
# print(labels)
# print(sil)

for min_sample_index, min_sample_size in enumerate(min_sample_range):
    for eps_index, eps in enumerate(epsilon_range):
#         clustering_algorithm = DBSCAN(eps=eps, min_samples=min_sample_size, metric='precomputed')
        clustering_algorithm = KMeans(n_clusters=min_sample_size)
        labels = clustering_algorithm.fit_predict(distance_matrix)
        print(len(set(labels)))  # TODO Change endline symbol
        cluster_count_matrix[eps_index][min_sample_index] = len(set(clustering_algorithm.labels_))
        if 1 < len(set(clustering_algorithm.labels_)) < number_of_paths:
            quality_matrix[eps_index][min_sample_index] = silhouette_score(np.array(coord_list[80:150]), labels)

# Plotting
plt.figure(figsize=(15, 5))
plt.imshow(quality_matrix, interpolation='none', aspect="auto",
           extent=[np.min(min_sample_range), np.max(min_sample_range), np.max(epsilon_range), np.min(epsilon_range)])
plt.clim(-1, 1)
plt.colorbar()

plt.savefig("dbscan_output/dbscan {}.png".format(int(time.time())), dpi=200, alpha=True)

plt.figure(figsize=(15, 5))
plt.imshow(cluster_count_matrix, aspect="auto", interpolation='none')
# plt.clim(-1, 1)
plt.colorbar()

In [ ]:
epsilon_range = np.arange(0.25, 1.55, 0.25) # y
leaf_size_range = np.arange(1, 20, 2) # x
quality_matrix = np.zeros((len(epsilon_range), len(leaf_size_range)))
cluster_count_matrix = np.zeros((len(epsilon_range), len(leaf_size_range)))

for leaf_size_index, leaf_size in enumerate(leaf_size_range):
    for eps_index, eps in enumerate(epsilon_range):
        clustering_algorithm = DBSCAN(eps=eps, min_samples=5, metric='precomputed')
        labels = clustering_algorithm.fit_predict(distance_matrix)
        cluster_count_matrix[eps_index][leaf_size_index] = len(set(clustering_algorithm.labels_))
        if len(set(clustering_algorithm.labels_)) > 1:
            quality_matrix[eps_index][leaf_size_index] = silhouette_score(np.array(coord_list), labels)

# Plotting
plt.figure(figsize=(15, 5))
plt.imshow(quality_matrix, interpolation='none', aspect="auto",
           extent=[np.min(leaf_size_range), np.max(leaf_size_range), np.max(epsilon_range), np.min(epsilon_range)])
plt.colorbar()
plt.figure(figsize=(15, 5))
plt.imshow(cluster_count_matrix, aspect="auto", interpolation='none')
plt.colorbar()